<a href="https://colab.research.google.com/github/Eitan177/Bioinformatics_Bootcamp/blob/main/informatics_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the GOAL Bioinformatics Bootcamp, 2024! To Setup this notebook please run the first thre cells. This will download the packages you will need to participate

In [ ]:
# @title Click the traingle to the left (hit control-enter if you would like to use your keyboard instead) to run this first cell

!pip install -q condacolab

In [ ]:
# @title Click the traingle to the left (hit control-enter if you would like to use your keyboard instead) to run this second cell
import condacolab
condacolab.install()
condacolab.check()

In [ ]:
# @title Click the traingle to the left (hit control-enter if you would like to use your keyboard instead) to run this third cell
!conda install bioconda::samtools
!conda install bioconda::sambamba
!conda install bioconda::fastp
!conda install bioconda::fastqc
!conda install bioconda::varscan==2.4.4
!conda install bioconda::bwa
!conda install bioconda::bwa
!conda install bioconda::abra2
!wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
!tar -zxvf annovar.latest.tar.gz
!rm -r annovar.latest.tar.gz
!rm -r annovar/humandb

Channels:
 - conda-forge
 - bioconda
Platform: linux-64